# Agent Memory: Building Memory-Enabled Agents with LangGraph

In this notebook, we'll explore **agent memory systems** - the ability for AI agents to remember information across interactions. We'll implement all five memory types from the **CoALA (Cognitive Architectures for Language Agents)** framework while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the 5 memory types from the CoALA framework
- Implement short-term memory with checkpointers and thread_id
- Build long-term memory with InMemoryStore and namespaces
- Use semantic memory for meaning-based retrieval
- Apply episodic memory for few-shot learning from past experiences
- Create procedural memory for self-improving agents
- Combine all memory types into a unified wellness agent

## Table of Contents:

- **Breakout Room #1:** Memory Foundations
  - Task 1: Dependencies
  - Task 2: Understanding Agent Memory (CoALA Framework)
  - Task 3: Short-Term Memory (MemorySaver, thread_id)
  - Task 4: Long-Term Memory (InMemoryStore, namespaces)
  - Task 5: Message Trimming & Context Management
  - Question #1 & Question #2
  - 🏗️ Activity #1: Store & Retrieve User Wellness Profile

- **Breakout Room #2:** Advanced Memory & Integration
  - Task 6: Semantic Memory (Embeddings + Search)
  - Task 7: Building Semantic Wellness Knowledge Base
  - Task 8: Episodic Memory (Few-Shot Learning)
  - Task 9: Procedural Memory (Self-Improving Agent)
  - Task 10: Unified Wellness Memory Agent
  - Question #3 & Question #4
  - 🏗️ Activity #2: Wellness Memory Dashboard

---
# 🤝 Breakout Room #1
## Memory Foundations

## Task 1: Dependencies

Before we begin, make sure you have:

1. **API Keys** for:
   - OpenAI (for GPT-4o-mini and embeddings)
   - LangSmith (optional, for tracing)

2. **Dependencies installed** via `uv sync`

In [2]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

In [6]:
# Set API Keys
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [7]:
# Optional: LangSmith for tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Agent Memory - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key (press Enter to skip): ") or ""

if not os.environ["LANGCHAIN_API_KEY"]:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")
else:
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")

LangSmith tracing enabled. Project: AIE9 - Agent Memory - 0b916022


In [8]:
# Initialize LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Test the connection
response = llm.invoke("Say 'Memory systems ready!' in exactly those words.")
print(response.content)

Memory systems ready!


## Task 2: Understanding Agent Memory (CoALA Framework)

The **CoALA (Cognitive Architectures for Language Agents)** framework identifies 5 types of memory that agents can use:

| Memory Type | Human Analogy | AI Implementation | Wellness Example |
|-------------|---------------|-------------------|------------------|
| **Short-term** | What someone just said | Conversation history within a thread | Current consultation conversation |
| **Long-term** | Remembering a friend's birthday | User preferences stored across sessions | User's goals, allergies, conditions |
| **Semantic** | Knowing Paris is in France | Facts retrieved by meaning | Wellness knowledge retrieval |
| **Episodic** | Remembering your first day at work | Learning from past experiences | Past successful advice patterns |
| **Procedural** | Knowing how to ride a bike | Self-improving instructions | Learned communication preferences |

### Memory Architecture Overview

```
┌─────────────────────────────────────────────────────────────────┐
│                    LangGraph Wellness Agent                     │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐           │
│  │  Short-term  │  │  Long-term   │  │   Semantic   │           │
│  │    Memory    │  │    Memory    │  │    Memory    │           │
│  │              │  │              │  │              │           │
│  │ Checkpointer │  │    Store     │  │Store+Embed   │           │
│  │ + thread_id  │  │ + namespace  │  │  + search()  │           │
│  └──────────────┘  └──────────────┘  └──────────────┘           │
│                                                                 │
│  ┌──────────────┐  ┌──────────────┐                             │
│  │   Episodic   │  │  Procedural  │                             │
│  │    Memory    │  │    Memory    │                             │
│  │              │  │              │                             │
│  │  Few-shot    │  │Self-modifying│                             │
│  │  examples    │  │   prompts    │                             │
│  └──────────────┘  └──────────────┘                             │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Key LangGraph Components

| Component | Memory Type | Scope |
|-----------|-------------|-------|
| `MemorySaver` (Checkpointer) | Short-term | Within a single thread |
| `InMemoryStore` | Long-term, Semantic, Episodic, Procedural | Across all threads |
| `thread_id` | Short-term | Identifies unique conversations |
| Namespaces | All store-based | Organizes memories by user/purpose |

**Documentation:**
- [CoALA Paper](https://arxiv.org/abs/2309.02427)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)

## Task 3: Short-Term Memory (MemorySaver, thread_id)

**Short-term memory** maintains context within a single conversation thread. Think of it like your working memory during a phone call - you remember what was said earlier, but once the call ends, those details fade.

In LangGraph, short-term memory is implemented through:
- **Checkpointer**: Saves the graph state at each step
- **thread_id**: Uniquely identifies each conversation

### How It Works

```
Thread 1: "Hi, I'm Alice"          Thread 2: "What's my name?"
     │                                   │
     ▼                                   ▼
┌──────────────┐                   ┌──────────────┐
│ Checkpointer │                   │ Checkpointer │
│  thread_1    │                   │  thread_2    │
│              │                   │              │
│ ["Hi Alice"] │                   │ [empty]      │
└──────────────┘                   └──────────────┘
     │                                   │
     ▼                                   ▼
"Hi Alice!"                        "I don't know your name"
```

In [9]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define the state schema for our graph
# The `add_messages` annotation tells LangGraph how to update the messages list
class State(TypedDict):
    messages: Annotated[list, add_messages]


# Define our wellness chatbot node
def wellness_chatbot(state: State):
    """Process the conversation and generate a wellness-focused response."""
    system_prompt = SystemMessage(content="""You are a friendly Personal Wellness Assistant. 
Help users with exercise, nutrition, sleep, and stress management questions.
Be supportive and remember details the user shares about themselves.""")
    
    messages = [system_prompt] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the graph
builder = StateGraph(State)
builder.add_node("chatbot", wellness_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

# Compile with a checkpointer for short-term memory
checkpointer = MemorySaver()
wellness_graph = builder.compile(checkpointer=checkpointer)

print("Wellness chatbot compiled with short-term memory (checkpointing)")

Wellness chatbot compiled with short-term memory (checkpointing)


In [6]:
# Test short-term memory within a thread
config = {"configurable": {"thread_id": "wellness_thread_1"}}

# First message - introduce ourselves
response = wellness_graph.invoke(
    {"messages": [HumanMessage(content="Hi! My name is Sarah and I want to improve my sleep.")]},
    config
)
print("User: Hi! My name is Sarah and I want to improve my sleep.")
print(f"Assistant: {response['messages'][-1].content}")
print()

User: Hi! My name is Sarah and I want to improve my sleep.
Assistant: Hi Sarah! It's great to meet you, and I'm glad you're focusing on improving your sleep. Sleep is so important for overall wellness. Can you tell me a bit more about your current sleep habits? For example, how many hours do you usually get, and do you have any specific challenges with your sleep?



In [7]:
# Second message - test if it remembers (same thread)
response = wellness_graph.invoke(
    {"messages": [HumanMessage(content="What's my name and what am I trying to improve?")]},
    config  # Same config = same thread_id
)
print("User: What's my name and what am I trying to improve?")
print(f"Assistant: {response['messages'][-1].content}")

User: What's my name and what am I trying to improve?
Assistant: Your name is Sarah, and you're trying to improve your sleep. If you have any specific challenges or questions about your sleep habits, feel free to share! I'm here to help.


In [8]:
# New thread - it won't remember Sarah!
different_config = {"configurable": {"thread_id": "wellness_thread_2"}}

response = wellness_graph.invoke(
    {"messages": [HumanMessage(content="What's my name?")]},
    different_config  # Different thread_id = no memory of Sarah
)
print("User (NEW thread): What's my name?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent doesn't know our name because this is a new thread!")

User (NEW thread): What's my name?
Assistant: I don't have your name yet! If you'd like to share it, I can remember it for our future conversations. How can I assist you today?

Notice: The agent doesn't know our name because this is a new thread!


In [9]:
# Inspect the state of thread 1
state = wellness_graph.get_state(config)
print(f"Thread 1 has {len(state.values['messages'])} messages:")
for msg in state.values['messages']:
    role = "User" if isinstance(msg, HumanMessage) else "Assistant"
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

Thread 1 has 4 messages:
  User: Hi! My name is Sarah and I want to improve my sleep.
  Assistant: Hi Sarah! It's great to meet you, and I'm glad you're focusing on improving your...
  User: What's my name and what am I trying to improve?
  Assistant: Your name is Sarah, and you're trying to improve your sleep. If you have any spe...


## Task 4: Long-Term Memory (InMemoryStore, namespaces)

**Long-term memory** stores information across different conversation threads. This is like remembering that your friend prefers tea over coffee - you remember it every time you meet them, regardless of what you're currently discussing.

In LangGraph, long-term memory uses:
- **Store**: A persistent key-value store
- **Namespaces**: Organize memories by user, application, or context

### Key Difference from Short-Term Memory

| Short-Term (Checkpointer) | Long-Term (Store) |
|---------------------------|-------------------|
| Scoped to a single thread | Shared across all threads |
| Automatic (messages) | Explicit (you decide what to store) |
| Conversation history | User preferences, facts, etc. |

In [10]:
from langgraph.store.memory import InMemoryStore

# Create a store for long-term memory
store = InMemoryStore()

# Namespaces organize memories - typically by user_id and category
user_id = "user_sarah"
profile_namespace = (user_id, "profile")
preferences_namespace = (user_id, "preferences")

# Store Sarah's wellness profile
store.put(profile_namespace, "name", {"value": "Sarah"})
store.put(profile_namespace, "goals", {"primary": "improve sleep", "secondary": "reduce stress"})
store.put(profile_namespace, "conditions", {"allergies": ["peanuts"], "injuries": ["bad knee"]})

# Store Sarah's preferences
store.put(preferences_namespace, "communication", {"style": "friendly", "detail_level": "moderate"})
store.put(preferences_namespace, "schedule", {"preferred_workout_time": "morning", "available_days": ["Mon", "Wed", "Fri"]})

print("Stored Sarah's profile and preferences in long-term memory")

Stored Sarah's profile and preferences in long-term memory


In [11]:
# Retrieve specific memories
name = store.get(profile_namespace, "name")
print(f"Name: {name.value}")

goals = store.get(profile_namespace, "goals")
print(f"Goals: {goals.value}")

# List all memories in a namespace
print("\nAll profile items:")
for item in store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Name: {'value': 'Sarah'}
Goals: {'primary': 'improve sleep', 'secondary': 'reduce stress'}

All profile items:
  name: {'value': 'Sarah'}
  goals: {'primary': 'improve sleep', 'secondary': 'reduce stress'}
  conditions: {'allergies': ['peanuts'], 'injuries': ['bad knee']}


In [11]:
from langgraph.store.base import BaseStore
from langchain_core.runnables import RunnableConfig

# Define state with user_id for personalization
class PersonalizedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def personalized_wellness_chatbot(state: PersonalizedState, config: RunnableConfig, *, store: BaseStore):
    """A wellness chatbot that uses long-term memory for personalization."""
    user_id = state["user_id"]
    profile_namespace = (user_id, "profile")
    preferences_namespace = (user_id, "preferences")
    
    # Retrieve user profile from long-term memory
    profile_items = list(store.search(profile_namespace))
    pref_items = list(store.search(preferences_namespace))
    
    # Build context from profile
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items])
    pref_text = "\n".join([f"- {p.key}: {p.value}" for p in pref_items])
    
    system_msg = f"""You are a Personal Wellness Assistant. You know the following about this user:

PROFILE:
{profile_text if profile_text else 'No profile stored.'}

PREFERENCES:
{pref_text if pref_text else 'No preferences stored.'}

Use this information to personalize your responses. Be supportive and helpful."""
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the personalized graph
builder2 = StateGraph(PersonalizedState)
builder2.add_node("chatbot", personalized_wellness_chatbot)
builder2.add_edge(START, "chatbot")
builder2.add_edge("chatbot", END)

# Compile with BOTH checkpointer (short-term) AND store (long-term)
personalized_graph = builder2.compile(
    checkpointer=MemorySaver(),
    store=store
)

print("Personalized graph compiled with both short-term and long-term memory")

Personalized graph compiled with both short-term and long-term memory


In [13]:
# Test the personalized chatbot - it knows Sarah's profile!
config = {"configurable": {"thread_id": "personalized_thread_1"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="What exercises would you recommend for me?")],
        "user_id": "user_sarah"
    },
    config
)

print("User: What exercises would you recommend for me?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: The agent knows about Sarah's bad knee without her mentioning it!")

User: What exercises would you recommend for me?
Assistant: Hi Sarah! It's great that you're looking to improve your wellness. Given your goal to reduce stress and your bad knee, I recommend focusing on low-impact exercises that can help you relax while being gentle on your joints. Here are a few options you might enjoy:

1. **Walking**: A simple and effective way to get moving. Try to find a scenic route to make it more enjoyable!

2. **Swimming**: This is fantastic for a full-body workout without putting stress on your knee. Plus, it's very refreshing!

3. **Yoga**: Gentle yoga can help with both stress reduction and flexibility. Look for classes that focus on restorative or yin yoga.

4. **Pilates**: This can strengthen your core and improve your overall stability, which is beneficial for your knee.

5. **Cycling**: Whether on a stationary bike or outdoors, cycling is a great way to get your heart rate up without straining your knee.

Since you prefer to work out in the morning and 

In [14]:
# Even in a NEW thread, it still knows Sarah's profile
# because long-term memory is cross-thread!

new_config = {"configurable": {"thread_id": "personalized_thread_2"}}

response = personalized_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you suggest a snack for me?")],
        "user_id": "user_sarah"
    },
    new_config
)

print("User (NEW thread): Can you suggest a snack for me?")
print(f"Assistant: {response['messages'][-1].content}")
print()
print("Notice: Even in a new thread, the agent knows Sarah has a peanut allergy!")

User (NEW thread): Can you suggest a snack for me?
Assistant: Absolutely, Sarah! Since you have a peanut allergy, let’s find something safe and delicious. How about some apple slices with almond butter? It’s a great combination of fiber and healthy fats, which can help keep you satisfied. Just make sure to choose almond butter that’s made in a peanut-free facility to avoid any cross-contamination. 

If you’re looking for something lighter, you could also try Greek yogurt with some berries. It’s packed with protein and antioxidants, which can be great for your overall wellness. Let me know if you’d like more options!

Notice: Even in a new thread, the agent knows Sarah has a peanut allergy!


## Task 5: Message Trimming & Context Management

Long conversations can exceed the LLM's context window. LangGraph provides utilities to manage message history:

- **`trim_messages`**: Keeps only recent messages up to a token limit
- **Summarization**: Compress older messages into summaries

### Why Trim Even with 128K Context?

Even with large context windows:
1. **Cost**: More tokens = higher API costs
2. **Latency**: Larger contexts take longer to process
3. **Quality**: Models can struggle with "lost in the middle" - important info buried in long contexts
4. **Relevance**: Old messages may not be relevant to current query

In [12]:
from langchain_core.messages import trim_messages

# Create a trimmer that keeps only recent messages
trimmer = trim_messages(
    max_tokens=500,  # Keep messages up to this token count
    strategy="last",  # Keep the most recent messages
    token_counter=llm,  # Use the LLM to count tokens
    include_system=True,  # Always keep system messages
    allow_partial=False,  # Don't cut messages in half
)

# Example: Create a long conversation
long_conversation = [
    SystemMessage(content="You are a wellness assistant."),
    HumanMessage(content="I want to improve my health."),
    AIMessage(content="Great goal! Let's start with exercise. What's your current activity level?"),
    HumanMessage(content="I walk about 30 minutes a day."),
    AIMessage(content="That's a good foundation. For cardiovascular health, aim for 150 minutes of moderate activity per week."),
    HumanMessage(content="What about nutrition?"),
    AIMessage(content="Focus on whole foods: vegetables, lean proteins, whole grains. Limit processed foods and added sugars."),
    HumanMessage(content="And sleep?"),
    AIMessage(content="Aim for 7-9 hours. Maintain a consistent sleep schedule and create a relaxing bedtime routine."),
    HumanMessage(content="What's the most important change I should make first?"),
]

# Trim to fit context window
trimmed = trimmer.invoke(long_conversation)
print(f"Original: {len(long_conversation)} messages")
print(f"Trimmed: {len(trimmed)} messages")
print("\nTrimmed conversation:")
for msg in trimmed:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:60] + "..." if len(msg.content) > 60 else msg.content
    print(f"  {role}: {content}")

Original: 10 messages
Trimmed: 10 messages

Trimmed conversation:
  System: You are a wellness assistant.
  Human: I want to improve my health.
  AI: Great goal! Let's start with exercise. What's your current a...
  Human: I walk about 30 minutes a day.
  AI: That's a good foundation. For cardiovascular health, aim for...
  Human: What about nutrition?
  AI: Focus on whole foods: vegetables, lean proteins, whole grain...
  Human: And sleep?
  AI: Aim for 7-9 hours. Maintain a consistent sleep schedule and ...
  Human: What's the most important change I should make first?


In [16]:
# Summarization approach for longer conversations

def summarize_conversation(messages: list, max_messages: int = 6) -> list:
    """Summarize older messages to manage context length."""
    if len(messages) <= max_messages:
        return messages
    
    # Keep the system message and last few messages
    system_msg = messages[0] if isinstance(messages[0], SystemMessage) else None
    content_messages = messages[1:] if system_msg else messages
    
    if len(content_messages) <= max_messages:
        return messages
    
    old_messages = content_messages[:-max_messages+1]
    recent_messages = content_messages[-max_messages+1:]
    
    # Summarize old messages
    summary_prompt = f"""Summarize this conversation in 2-3 sentences, 
capturing key wellness topics discussed and any important user information:

{chr(10).join([f'{type(m).__name__}: {m.content[:200]}' for m in old_messages])}"""
    
    summary = llm.invoke(summary_prompt)
    
    # Return: system + summary + recent messages
    result = []
    if system_msg:
        result.append(system_msg)
    result.append(SystemMessage(content=f"[Previous conversation summary: {summary.content}]"))
    result.extend(recent_messages)
    
    return result


# Test summarization
summarized = summarize_conversation(long_conversation, max_messages=4)
print(f"Summarized: {len(summarized)} messages")
print("\nSummarized conversation:")
for msg in summarized:
    role = type(msg).__name__.replace("Message", "")
    content = msg.content[:80] + "..." if len(msg.content) > 80 else msg.content
    print(f"  {role}: {content}")

Summarized: 5 messages

Summarized conversation:
  System: You are a wellness assistant.
  System: [Previous conversation summary: The conversation centers around the user's goal ...
  Human: And sleep?
  AI: Aim for 7-9 hours. Maintain a consistent sleep schedule and create a relaxing be...
  Human: What's the most important change I should make first?


---
## ❓ Question #1:

What are the trade-offs between **short-term memory** (checkpointer) vs **long-term memory** (store)? When should wellness data move from short-term to long-term?

Consider:
- What information should persist across sessions?
- What are the privacy implications of each?
- How would you decide what to promote from short-term to long-term?

##### Answer:
Trade-offs Between Short-term and Long-term Memory Systems
When working with AI systems that manage user information, it’s important to understand the trade-offs between short-term memory (often called the checkpointer) and long-term memory (the store). Both serve different purposes, and choosing when to use each one affects context, privacy, and system behavior.

Short-term Memory (Checkpointer)
Short-term memory only exists within the current conversation thread. It updates automatically with every message, which makes it useful for maintaining conversational flow. Because it expires when the session ends, it carries lower long-term privacy risks. However, its size grows as the conversation continues, and it can include sensitive information if the user mentions it during the session.

Long-term Memory (Store)
Long-term memory persists across threads and even across sessions. Unlike the checkpointer, it is managed manually, meaning the system only stores what the user explicitly wants to keep. This memory is ideal for stable personal information, such as identity details or consistent preferences. Since it is persistent, it requires stronger privacy controls, including explicit consent, retention policies, encryption, and the ability to delete information in compliance with regulations like GDPR.

When to Move Information from Short-term to Long-term
Promoting information to long-term memory makes sense when it is stable, important, or safety‑related. Examples include:

Permanent identity details (name, age, chronic medical conditions)
Explicit user preferences (“Always answer briefly”, “I’m vegetarian”)
Safety‑critical information (allergies, injuries, contraindications)
Any detail the user explicitly asks the system to remember (“Remember that I have diabetes”)

Privacy Considerations
Short-term memory is temporary, which reduces long-term exposure but still may contain sensitive data during the session. Long-term memory, on the other hand, carries higher privacy risks because it persists. For that reason, it requires:

Explicit user consent
Clear retention and deletion policies
Encryption and access control
Support for the right to be forgotten

## ❓ Question #2:

Why use message trimming with a 128K context window when HealthWellnessGuide.txt is only ~16KB? What should **always** be preserved when trimming a wellness consultation?

Consider:
- The "lost in the middle" phenomenon
- Cost and latency implications
- What user information is critical for safety (allergies, conditions, etc.)

##### Answer:
Why Should We Do Trimming Even When 128K Context Is Available?
When working with large language models, it might seem like having a 128K‑token context window means we can just keep everything. But in practice, trimming the context is still important for several reasons.

1. The “Lost in the Middle” Phenomenon
LLMs tend to be less accurate with information placed in the middle of very long contexts.
They pay more attention to the beginning and the end.
Keeping the context shorter helps the model focus on the most important details.
So even if we can load 128K tokens, it doesn’t mean the model will use them effectively.

2. Cost and Latency
Using 128K tokens is much more expensive than using 16K.
In fact, 128K can cost around 8 times more.
Processing time also increases almost linearly with context length.
For long conversations, this becomes expensive and slow very quickly.

3. Relevance
Most of the time, messages from 40 or 50 turns ago are no longer relevant to the current question.
Keeping only the recent and meaningful context improves answer quality.
Old or irrelevant information can confuse the model instead of helping it.

What Should ALWAYS Be Preserved?
In a wellness‑related consultation, there are certain elements that should never be trimmed because they are essential for safety and correct reasoning.
1. System Message
The agent’s core instructions must always remain intact.
2. Critical Safety Information
This includes:
Known allergies (food or medication)
Chronic medical conditions (like diabetes or hypertension)
Physical injuries or limitations
Current medications (to avoid dangerous interactions)
3. Explicit Contraindications
4. The User’s Current Goal
The specific problem the user is trying to solve must always stay in context.

---
## 🏗️ Activity #1: Store & Retrieve User Wellness Profile

Build a complete wellness profile system that:
1. Defines a wellness profile schema (name, goals, conditions, preferences)
2. Creates functions to store and retrieve profile data
3. Builds a personalized wellness agent that uses the profile
4. Tests that different users get different advice

### Requirements:
- Define at least 5 profile attributes
- Support multiple users with different profiles
- Agent should reference profile data in responses

In [ ]:
# ============================================================================
# Activity #1: Store & Retrieve User Wellness Profile
# ============================================================================

# Define Wellness Profile Schema
# ----------------------------------------

WELLNESS_PROFILE_SCHEMA = {
    "name": str,                    # Nombre del usuario
    "age": int,                     # Edad para recomendaciones apropiadas
    "goals": list,                  # Objetivos wellness (ej: ["improve sleep", "lose weight"])
    "conditions": list,             # Condiciones médicas (ej: ["diabetes", "hypertension"])
    "allergies": list,              # Alergias alimentarias/medicamentosas
    "fitness_level": str,           # "beginner", "intermediate", "advanced"
    "preferred_activities": list,   # Preferencias de ejercicio
    "schedule": dict,               # Días/horarios disponibles
    "dietary_preferences": list,    # "vegetarian", "vegan", "gluten-free", etc.
}

In [17]:
#  Helper Functions for Profile Management
# ------------------------------------------------

def store_wellness_profile(store, user_id: str, profile: dict):
    """Store a user's wellness profile in long-term memory."""
    from datetime import datetime
    
    profile_namespace = (user_id, "profile")
    
    # Store each attribute separately for easier updates
    for key, value in profile.items():
        store.put(
            profile_namespace, 
            key, 
            {"value": value, "updated_at": datetime.now().isoformat()}
        )
    
    print(f"✓ Stored profile for user: {user_id}")


def get_wellness_profile(store, user_id: str) -> dict:
    """Retrieve a user's complete wellness profile."""
    profile_namespace = (user_id, "profile")
    
    # Retrieve all items in the profile namespace
    profile_items = list(store.search(profile_namespace))
    
    if not profile_items:
        return {}
    
    # Reconstruct the profile dictionary
    profile = {}
    for item in profile_items:
        profile[item.key] = item.value.get("value")
    
    return profile


def update_wellness_profile(store, user_id: str, updates: dict):
    """Update specific fields in a user's wellness profile."""
    from datetime import datetime
    
    profile_namespace = (user_id, "profile")
    
    for key, value in updates.items():
        store.put(
            profile_namespace, 
            key, 
            {"value": value, "updated_at": datetime.now().isoformat()}
        )
    
    print(f"✓ Updated {len(updates)} field(s) for user: {user_id}")


def format_profile_for_prompt(profile: dict) -> str:
    """Format profile data for inclusion in system prompt."""
    if not profile:
        return "No profile information available."
    
    lines = ["USER PROFILE:"]
    lines.append(f"- Name: {profile.get('name', 'Unknown')}")
    lines.append(f"- Age: {profile.get('age', 'Unknown')}")
    lines.append(f"- Fitness Level: {profile.get('fitness_level', 'Unknown')}")
    
    if profile.get('goals'):
        lines.append(f"- Goals: {', '.join(profile['goals'])}")
    
    if profile.get('conditions'):
        lines.append(f"- Medical Conditions: {', '.join(profile['conditions'])}")
    
    if profile.get('allergies'):
        lines.append(f"- Allergies: {', '.join(profile['allergies'])}")
    
    if profile.get('preferred_activities'):
        lines.append(f"- Preferred Activities: {', '.join(profile['preferred_activities'])}")
    
    if profile.get('dietary_preferences'):
        lines.append(f"- Dietary Preferences: {', '.join(profile['dietary_preferences'])}")
    
    if profile.get('schedule'):
        schedule = profile['schedule']
        lines.append(f"- Available Days: {', '.join(schedule.get('days', []))}")
        lines.append(f"- Preferred Time: {schedule.get('preferred_time', 'Any')}")
    
    return "\n".join(lines)



In [19]:
#  Create Sample User Profiles
# ------------------------------------

# Profile 1: Sarah - Beginner, vegetarian, wants better sleep
sarah_profile = {
    "name": "Sarah",
    "age": 32,
    "goals": ["improve sleep", "reduce stress"],
    "conditions": ["mild anxiety"],
    "allergies": ["peanuts"],
    "fitness_level": "beginner",
    "preferred_activities": ["yoga", "walking", "swimming"],
    "schedule": {
        "days": ["Monday", "Wednesday", "Friday"],
        "preferred_time": "morning"
    },
    "dietary_preferences": ["vegetarian"]
}

# Profile 2: Mike - Intermediate, diabetic, wants to lose weight
mike_profile = {
    "name": "Mike",
    "age": 45,
    "goals": ["lose weight", "build muscle", "improve cardiovascular health"],
    "conditions": ["type 2 diabetes", "high blood pressure"],
    "allergies": ["shellfish"],
    "fitness_level": "intermediate",
    "preferred_activities": ["cycling", "weightlifting", "hiking"],
    "schedule": {
        "days": ["Tuesday", "Thursday", "Saturday", "Sunday"],
        "preferred_time": "evening"
    },
    "dietary_preferences": ["low-carb", "high-protein"]
}

# Profile 3: Emma - Advanced, vegan, wants more energy
emma_profile = {
    "name": "Emma",
    "age": 28,
    "goals": ["increase energy", "improve flexibility"],
    "conditions": [],
    "allergies": ["dairy"],
    "fitness_level": "advanced",
    "preferred_activities": ["running", "pilates", "rock climbing"],
    "schedule": {
        "days": ["Monday", "Tuesday", "Thursday", "Friday"],
        "preferred_time": "morning"
    },
    "dietary_preferences": ["vegan", "gluten-free"]
}

# Store all profiles
store_wellness_profile(store, "user_sarah", sarah_profile)
store_wellness_profile(store, "user_mike", mike_profile)
store_wellness_profile(store, "user_emma", emma_profile)


✓ Stored profile for user: user_sarah
✓ Stored profile for user: user_mike
✓ Stored profile for user: user_emma


In [21]:
# Build Personalized Wellness Agent
# ------------------------------------------

class WellnessState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def wellness_agent_with_profile(state: WellnessState, config: RunnableConfig, *, store: BaseStore):
    """Wellness agent that uses stored profiles for personalization."""
    
    user_id = state["user_id"]
    
    # Retrieve user profile from long-term memory
    profile = get_wellness_profile(store, user_id)
    profile_context = format_profile_for_prompt(profile)
    
    # Build personalized system prompt
    system_prompt = f"""You are a Personal Wellness Assistant specialized in exercise, nutrition, sleep, and stress management.

{profile_context}

GUIDELINES:
- Always personalize advice based on the user's profile
- Consider their fitness level when recommending exercises
- Avoid recommending foods they're allergic to
- Respect their dietary preferences
- Suggest activities that match their schedule and preferences
- Be mindful of their medical conditions when giving advice
- Focus on helping them achieve their stated goals
- Be supportive, encouraging, and practical

Reference specific profile details in your responses to show personalization."""

    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    
    return {"messages": [response]}


# Build and compile the graph
wellness_builder = StateGraph(WellnessState)
wellness_builder.add_node("agent", wellness_agent_with_profile)
wellness_builder.add_edge(START, "agent")
wellness_builder.add_edge("agent", END)

wellness_graph = wellness_builder.compile(
    checkpointer=MemorySaver(),
    store=store
)

print("✓ Personalized wellness agent compiled!")


✓ Personalized wellness agent compiled!


In [22]:
# Test with Different Users
# ----------------------------------

print("\n" + "="*70)
print("TEST 1: Sarah (Beginner, Vegetarian, Sleep Goals)")
print("="*70)

config1 = {"configurable": {"thread_id": "sarah_test_1"}}
response1 = wellness_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you recommend a workout routine and dinner for me?")],
        "user_id": "user_sarah"
    },
    config1
)
print("\nSarah's Query: Can you recommend a workout routine and dinner for me?")
print(f"\nAgent Response:\n{response1['messages'][-1].content}")


print("\n" + "="*70)
print("TEST 2: Mike (Intermediate, Diabetic, Weight Loss)")
print("="*70)

config2 = {"configurable": {"thread_id": "mike_test_1"}}
response2 = wellness_graph.invoke(
    {
        "messages": [HumanMessage(content="What exercises should I do this week and what should I eat?")],
        "user_id": "user_mike"
    },
    config2
)
print("\nMike's Query: What exercises should I do this week and what should I eat?")
print(f"\nAgent Response:\n{response2['messages'][-1].content}")


print("\n" + "="*70)
print("TEST 3: Emma (Advanced, Vegan, Energy Goals)")
print("="*70)

config3 = {"configurable": {"thread_id": "emma_test_1"}}
response3 = wellness_graph.invoke(
    {
        "messages": [HumanMessage(content="I'm feeling low on energy lately. What would you suggest?")],
        "user_id": "user_emma"
    },
    config3
)
print("\nEmma's Query: I'm feeling low on energy lately. What would you suggest?")
print(f"\nAgent Response:\n{response3['messages'][-1].content}")


# Verify profiles are correctly stored
print("\n" + "="*70)
print("VERIFICATION: Stored Profiles")
print("="*70)

for user_id in ["user_sarah", "user_mike", "user_emma"]:
    profile = get_wellness_profile(store, user_id)
    print(f"\n{user_id}:")
    print(format_profile_for_prompt(profile))


# Test profile update
print("\n" + "="*70)
print("TEST: Profile Update")
print("="*70)

print("\nUpdating Sarah's fitness level...")
update_wellness_profile(store, "user_sarah", {"fitness_level": "intermediate"})

updated_profile = get_wellness_profile(store, "user_sarah")
print(f"New fitness level: {updated_profile.get('fitness_level')}")

print("\n✓ Activity #1 Complete!")



TEST 1: Sarah (Beginner, Vegetarian, Sleep Goals)

Sarah's Query: Can you recommend a workout routine and dinner for me?

Agent Response:
Absolutely, Sarah! Based on your profile, I’ll create a beginner-friendly workout routine and a delicious vegetarian dinner that aligns with your goals of improving sleep and reducing stress.

### Workout Routine (Monday, Wednesday, Friday - Morning)

**Monday: Yoga for Relaxation**
- **Duration:** 30 minutes
- **Focus:** Gentle stretches and breathing exercises
- **Suggested Poses:** Child’s Pose, Cat-Cow, Downward Dog, Seated Forward Bend, and Savasana (Corpse Pose) for relaxation.

**Wednesday: Walking**
- **Duration:** 30-45 minutes
- **Focus:** Enjoy a brisk walk in a park or around your neighborhood. This will help clear your mind and reduce stress while also improving your cardiovascular health.

**Friday: Swimming**
- **Duration:** 30 minutes
- **Focus:** Gentle laps or water aerobics. Swimming is great for relaxation and can help improve yo

In [17]:
### YOUR CODE HERE ###

# Step 1: Define a wellness profile schema
# Example attributes: name, age, goals, conditions, allergies, fitness_level, preferred_activities


# Step 2: Create helper functions to store and retrieve profiles
def store_wellness_profile(store, user_id: str, profile: dict):
    """Store a user's wellness profile."""
    pass


def get_wellness_profile(store, user_id: str) -> dict:
    """Retrieve a user's wellness profile."""
    pass


# Step 3: Create two different user profiles


# Step 4: Build a personalized agent that uses profiles


# Step 5: Test with different users - they should get different advice


---
# 🤝 Breakout Room #2
## Advanced Memory & Integration

## Task 6: Semantic Memory (Embeddings + Search)

**Semantic memory** stores facts and retrieves them based on *meaning* rather than exact matches. This is like how you might remember "that restaurant with the great pasta" even if you can't remember its exact name.

In LangGraph, semantic memory uses:
- **Store with embeddings**: Converts text to vectors for similarity search
- **`store.search()`**: Finds relevant memories by semantic similarity

### How It Works

```
User asks: "What helps with headaches?"
         ↓
Query embedded → [0.2, 0.8, 0.1, ...]
         ↓
Compare with stored wellness facts:
  - "Hydration can relieve headaches" → 0.92 similarity ✓
  - "Exercise improves sleep" → 0.35 similarity
         ↓
Return: "Hydration can relieve headaches"
```

In [13]:
from langchain_openai import OpenAIEmbeddings

# Create embeddings model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Create a store with semantic search enabled
semantic_store = InMemoryStore(
    index={
        "embed": embeddings,
        "dims": 1536,  # Dimension of text-embedding-3-small
    }
)

print("Semantic memory store created with embedding support")

Semantic memory store created with embedding support


In [19]:
# Store various wellness facts as semantic memories
namespace = ("wellness", "facts")

wellness_facts = [
    ("fact_1", {"text": "Drinking water can help relieve headaches caused by dehydration"}),
    ("fact_2", {"text": "Regular exercise improves sleep quality and helps you fall asleep faster"}),
    ("fact_3", {"text": "Deep breathing exercises can reduce stress and anxiety within minutes"}),
    ("fact_4", {"text": "Eating protein at breakfast helps maintain steady energy levels throughout the day"}),
    ("fact_5", {"text": "Blue light from screens can disrupt your circadian rhythm and sleep"}),
    ("fact_6", {"text": "Walking for 30 minutes daily can improve cardiovascular health"}),
    ("fact_7", {"text": "Magnesium-rich foods like nuts and leafy greens can help with muscle cramps"}),
    ("fact_8", {"text": "A consistent sleep schedule, even on weekends, improves overall sleep quality"}),
]

for key, value in wellness_facts:
    semantic_store.put(namespace, key, value)

print(f"Stored {len(wellness_facts)} wellness facts in semantic memory")

Stored 8 wellness facts in semantic memory


In [20]:
# Search semantically - notice we don't need exact matches!

queries = [
    "My head hurts, what should I do?",
    "How can I get better rest at night?",
    "I'm feeling stressed and anxious",
    "What should I eat in the morning?",
]

for query in queries:
    print(f"\nQuery: {query}")
    results = semantic_store.search(namespace, query=query, limit=2)
    for r in results:
        print(f"   {r.value['text']} (score: {r.score:.3f})")


Query: My head hurts, what should I do?
   Drinking water can help relieve headaches caused by dehydration (score: 0.327)
   Magnesium-rich foods like nuts and leafy greens can help with muscle cramps (score: 0.173)

Query: How can I get better rest at night?
   Regular exercise improves sleep quality and helps you fall asleep faster (score: 0.463)
   A consistent sleep schedule, even on weekends, improves overall sleep quality (score: 0.426)

Query: I'm feeling stressed and anxious
   Deep breathing exercises can reduce stress and anxiety within minutes (score: 0.415)
   Drinking water can help relieve headaches caused by dehydration (score: 0.224)

Query: What should I eat in the morning?
   Eating protein at breakfast helps maintain steady energy levels throughout the day (score: 0.467)
   Walking for 30 minutes daily can improve cardiovascular health (score: 0.249)


## Task 7: Building Semantic Wellness Knowledge Base

Let's load the HealthWellnessGuide.txt and create a semantic knowledge base that our agent can search.

This is similar to RAG from Session 4, but now using LangGraph's Store API instead of a separate vector database.

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load and chunk the wellness document
loader = TextLoader("data/HealthWellnessGuide.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(documents)

print(f"Loaded and split into {len(chunks)} chunks")
print(f"\nSample chunk:\n{chunks[0].page_content[:200]}...")

Loaded and split into 45 chunks

Sample chunk:
The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things yo...


In [22]:
# Store chunks in semantic memory
knowledge_namespace = ("wellness", "knowledge")

for i, chunk in enumerate(chunks):
    semantic_store.put(
        knowledge_namespace,
        f"chunk_{i}",
        {"text": chunk.page_content, "source": "HealthWellnessGuide.txt"}
    )

print(f"Stored {len(chunks)} chunks in semantic knowledge base")

Stored 45 chunks in semantic knowledge base


In [15]:
# Build a semantic search wellness chatbot

class SemanticState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def semantic_wellness_chatbot(state: SemanticState, config: RunnableConfig, *, store: BaseStore):
    """A wellness chatbot that retrieves relevant facts using semantic search."""
    user_message = state["messages"][-1].content
    
    # Search for relevant knowledge
    knowledge_results = store.search(
        ("wellness", "knowledge"),
        query=user_message,
        limit=3
    )
    
    # Build context from retrieved knowledge
    if knowledge_results:
        knowledge_text = "\n\n".join([f"- {r.value['text']}" for r in knowledge_results])
        system_msg = f"""You are a Personal Wellness Assistant with access to a wellness knowledge base.

Relevant information from your knowledge base:
{knowledge_text}

Use this information to answer the user's question. If the information doesn't directly answer their question, use your general knowledge but mention what you found."""
    else:
        system_msg = "You are a Personal Wellness Assistant. Answer wellness questions helpfully."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build and compile
builder3 = StateGraph(SemanticState)
builder3.add_node("chatbot", semantic_wellness_chatbot)
builder3.add_edge(START, "chatbot")
builder3.add_edge("chatbot", END)

semantic_graph = builder3.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Semantic wellness chatbot ready")

Semantic wellness chatbot ready


In [24]:
# Test semantic retrieval
config = {"configurable": {"thread_id": "semantic_thread_1"}}

questions = [
    "What exercises can help with lower back pain?",
    "How can I improve my sleep quality?",
    "What should I eat for better gut health?",
]

for q in questions:
    response = semantic_graph.invoke(
        {"messages": [HumanMessage(content=q)], "user_id": "test_user"},
        config
    )
    print(f"\nUser: {q}")
    print(f"Assistant: {response['messages'][-1].content[:500]}...")


User: What exercises can help with lower back pain?
Assistant: There are several effective exercises that can help alleviate lower back pain. Here are some recommended ones:

1. **Cat-Cow Stretch**: Start on your hands and knees. Alternate between arching your back up (like a cat) and letting it sag down (like a cow). Perform 10-15 repetitions.

2. **Bird Dog**: From a hands-and-knees position, extend your opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Aim for 10 repetitions per side.

3. **Partial Crunches**: L...

User: How can I improve my sleep quality?
Assistant: Improving your sleep quality can be achieved through several essential sleep hygiene practices and creating an optimal sleep environment. Here are some tips:

### Sleep Hygiene Practices:
1. **Maintain a Consistent Sleep Schedule**: Go to bed and wake up at the same time every day, even on weekends.
2. **Create a Relaxing Bedtime Routine**: Engage in calming activities before

## Task 8: Episodic Memory (Few-Shot Learning)

**Episodic memory** stores past experiences and uses them as examples for future tasks. In humans, this is like remembering the first time you successfully gave someone good advice and using that experience to guide future interactions.

For AI agents, episodic memory is often implemented as **few-shot examples** - showing the model examples of correct behavior so it can learn patterns.

### The CoALA Framework Perspective

> "Facts can be written to semantic memory, whereas **experiences** can be written to episodic memory."

Key insight: Sometimes it's easier to **show** than **tell**. LLMs learn well from examples!

In [25]:
# Create episodic memory with past successful interactions
episodic_namespace = ("agent", "episodes")

successful_episodes = [
    {
        "situation": "User asked about starting exercise with joint pain",
        "input": "I want to start exercising but my knees hurt. What should I do?",
        "output": """For knee-friendly exercise, I'd recommend starting with low-impact activities:

1. **Swimming or water aerobics** - Water supports your joints while providing great cardio
2. **Cycling** - Either stationary or regular bikes are gentle on knees
3. **Elliptical machine** - Provides cardio without impact

Start slowly with 15-20 minutes and gradually increase. If you experience pain, stop and consult a healthcare provider. Would you like specific exercises for any of these?""",
        "feedback": "User appreciated the specific alternatives and safety advice"
    },
    {
        "situation": "User struggling with sleep and stress",
        "input": "I can't fall asleep because I'm always thinking about work. Help!",
        "output": """Racing thoughts at bedtime is common, especially with work stress. Here's a proven approach:

**Before bed:**
- Write down tomorrow's to-do list (gets thoughts out of your head)
- Set a "worry time" earlier in the evening - not at bedtime

**At bedtime:**
- Try the 4-7-8 breathing technique: Inhale 4 counts, hold 7, exhale 8
- Use the "cognitive shuffle" - think of random, unrelated objects (apple, guitar, mountain...)

Which of these would you like to try first?""",
        "feedback": "User found the structured approach and specific techniques helpful"
    },
    {
        "situation": "User asking about healthy eating on a budget",
        "input": "How can I eat healthy when I don't have much money?",
        "output": """Eating healthy on a budget is absolutely possible! Here are practical strategies:

**Budget-friendly nutritious foods:**
- Eggs, beans, lentils (cheap protein)
- Frozen vegetables (same nutrition as fresh, longer lasting)
- Oats, rice, whole grain bread
- Canned fish (tuna, salmon)

**Money-saving tips:**
- Buy in bulk when possible
- Plan meals around sales
- Cook in batches and freeze portions

What's your typical weekly food budget? I can help you create a specific meal plan.""",
        "feedback": "User valued the practical, actionable advice without judgment"
    },
]

for i, episode in enumerate(successful_episodes):
    semantic_store.put(
        episodic_namespace,
        f"episode_{i}",
        {
            "text": episode["situation"],  # Used for semantic search
            **episode
        }
    )

print(f"Stored {len(successful_episodes)} episodic memories (past successful interactions)")

Stored 3 episodic memories (past successful interactions)


In [26]:
class EpisodicState(TypedDict):
    messages: Annotated[list, add_messages]


def episodic_wellness_chatbot(state: EpisodicState, config: RunnableConfig, *, store: BaseStore):
    """A chatbot that learns from past successful interactions."""
    user_question = state["messages"][-1].content
    
    # Search for similar past experiences
    similar_episodes = store.search(
        ("agent", "episodes"),
        query=user_question,
        limit=1
    )
    
    # Build few-shot examples from past episodes
    if similar_episodes:
        episode = similar_episodes[0].value
        few_shot_example = f"""Here's an example of a similar wellness question I handled well:

User asked: {episode['input']}

My response was:
{episode['output']}

The user feedback was: {episode['feedback']}

Use this as inspiration for the style, structure, and tone of your response, but tailor it to the current question."""
        
        system_msg = f"""You are a Personal Wellness Assistant. Learn from your past successes:

{few_shot_example}"""
    else:
        system_msg = "You are a Personal Wellness Assistant. Be helpful, specific, and supportive."
    
    messages = [SystemMessage(content=system_msg)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build the episodic memory graph
builder4 = StateGraph(EpisodicState)
builder4.add_node("chatbot", episodic_wellness_chatbot)
builder4.add_edge(START, "chatbot")
builder4.add_edge("chatbot", END)

episodic_graph = builder4.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Episodic memory chatbot ready")

Episodic memory chatbot ready


In [27]:
# Test episodic memory - similar question to stored episode
config = {"configurable": {"thread_id": "episodic_thread_1"}}

response = episodic_graph.invoke(
    {"messages": [HumanMessage(content="I want to exercise more but I have a bad hip. What can I do?")]},
    config
)

print("User: I want to exercise more but I have a bad hip. What can I do?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The response structure mirrors the successful knee pain episode!")

User: I want to exercise more but I have a bad hip. What can I do?

Assistant: It's great that you want to exercise more! For a bad hip, focusing on low-impact activities can help you stay active while minimizing discomfort. Here are some options to consider:

1. **Swimming or water aerobics** - The buoyancy of water reduces stress on your hip joints while providing a full-body workout.
2. **Cycling** - Using a stationary bike or cycling outdoors can be gentle on your hips and still give you a good cardiovascular workout.
3. **Walking** - Start with short, flat walks and gradually increase your distance. Consider using supportive shoes and walking on softer surfaces like grass or tracks.
4. **Yoga or Pilates** - These practices can improve flexibility and strength without putting too much strain on your hips. Look for classes that focus on gentle movements.
5. **Resistance training** - Focus on upper body and core exercises that don’t involve your hips too much. You can use resistance 

## Task 9: Procedural Memory (Self-Improving Agent)

**Procedural memory** stores the rules and instructions that guide behavior. In humans, this is like knowing *how* to give good advice - it's internalized knowledge about performing tasks.

For AI agents, procedural memory often means **self-modifying prompts**. The agent can:
1. Store its current instructions in the memory store
2. Reflect on feedback from interactions
3. Update its own instructions to improve

### The Reflection Pattern

```
User feedback: "Your advice is too long and complicated"
         ↓
Agent reflects on current instructions
         ↓
Agent updates instructions: "Keep advice concise and actionable"
         ↓
Future responses use updated instructions
```

In [28]:
# Initialize procedural memory with base instructions
procedural_namespace = ("agent", "instructions")

initial_instructions = """You are a Personal Wellness Assistant.

Guidelines:
- Be supportive and non-judgmental
- Provide evidence-based wellness information
- Ask clarifying questions when needed
- Encourage healthy habits without being preachy"""

semantic_store.put(
    procedural_namespace,
    "wellness_assistant",
    {"instructions": initial_instructions, "version": 1}
)

print("Initialized procedural memory with base instructions")
print(f"\nCurrent Instructions (v1):\n{initial_instructions}")

Initialized procedural memory with base instructions

Current Instructions (v1):
You are a Personal Wellness Assistant.

Guidelines:
- Be supportive and non-judgmental
- Provide evidence-based wellness information
- Ask clarifying questions when needed
- Encourage healthy habits without being preachy


In [29]:
class ProceduralState(TypedDict):
    messages: Annotated[list, add_messages]
    feedback: str  # Optional feedback from user


def get_instructions(store: BaseStore) -> tuple[str, int]:
    """Retrieve current instructions from procedural memory."""
    item = store.get(("agent", "instructions"), "wellness_assistant")
    if item is None:
        return "You are a helpful wellness assistant.", 0
    return item.value["instructions"], item.value["version"]


def procedural_assistant_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Respond using current procedural instructions."""
    instructions, version = get_instructions(store)
    
    messages = [SystemMessage(content=instructions)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


def reflection_node(state: ProceduralState, config: RunnableConfig, *, store: BaseStore):
    """Reflect on feedback and update instructions if needed."""
    feedback = state.get("feedback", "")
    
    if not feedback:
        return {}  # No feedback, no update needed
    
    # Get current instructions
    current_instructions, version = get_instructions(store)
    
    # Ask the LLM to reflect and improve instructions
    reflection_prompt = f"""You are improving a wellness assistant's instructions based on user feedback.

Current Instructions:
{current_instructions}

User Feedback:
{feedback}

Based on this feedback, provide improved instructions. Keep the same general format but incorporate the feedback.
Only output the new instructions, nothing else."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    new_instructions = response.content
    
    # Update procedural memory with new instructions
    store.put(
        ("agent", "instructions"),
        "wellness_assistant",
        {"instructions": new_instructions, "version": version + 1}
    )
    
    print(f"\nInstructions updated to version {version + 1}")
    return {}


def should_reflect(state: ProceduralState) -> str:
    """Decide whether to reflect on feedback."""
    if state.get("feedback"):
        return "reflect"
    return "end"


# Build the procedural memory graph
builder5 = StateGraph(ProceduralState)
builder5.add_node("assistant", procedural_assistant_node)
builder5.add_node("reflect", reflection_node)

builder5.add_edge(START, "assistant")
builder5.add_conditional_edges("assistant", should_reflect, {"reflect": "reflect", "end": END})
builder5.add_edge("reflect", END)

procedural_graph = builder5.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Procedural memory graph ready (with self-improvement capability)")

Procedural memory graph ready (with self-improvement capability)


In [30]:
# Test with initial instructions
config = {"configurable": {"thread_id": "procedural_thread_1"}}

response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How can I reduce stress?")],
        "feedback": ""  # No feedback yet
    },
    config
)

print("User: How can I reduce stress?")
print(f"\nAssistant (v1 instructions):\n{response['messages'][-1].content}")

User: How can I reduce stress?

Assistant (v1 instructions):
Reducing stress is a great goal, and there are several effective strategies you can try. Here are some evidence-based methods:

1. **Mindfulness and Meditation**: Practicing mindfulness or meditation can help you stay present and reduce anxiety. Even a few minutes a day can make a difference.

2. **Physical Activity**: Regular exercise is a powerful stress reliever. It can boost your mood and improve your overall health. Find an activity you enjoy, whether it's walking, dancing, or yoga.

3. **Deep Breathing**: Simple deep breathing exercises can help calm your mind and body. Try inhaling deeply through your nose, holding for a few seconds, and exhaling slowly through your mouth.

4. **Connect with Others**: Talking to friends or family can provide support and help you feel less isolated. Sometimes just sharing your feelings can lighten the load.

5. **Time Management**: Organizing your tasks and setting priorities can help y

In [31]:
# Now provide feedback - the agent will update its own instructions!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How can I reduce stress?")],
        "feedback": "Your responses are too long. Please be more concise and give me 3 actionable tips maximum."
    },
    {"configurable": {"thread_id": "procedural_thread_2"}}
)


Instructions updated to version 2


In [32]:
# Check the updated instructions
new_instructions, version = get_instructions(semantic_store)
print(f"Updated Instructions (v{version}):\n")
print(new_instructions)

Updated Instructions (v2):

You are a Personal Wellness Assistant.

Guidelines:
- Be supportive and non-judgmental
- Provide evidence-based wellness information
- Ask clarifying questions when needed
- Encourage healthy habits without being preachy
- Keep responses concise, offering a maximum of 3 actionable tips.


In [33]:
# Test with updated instructions - should be more concise now!
response = procedural_graph.invoke(
    {
        "messages": [HumanMessage(content="How can I sleep better?")],
        "feedback": ""  # No feedback this time
    },
    {"configurable": {"thread_id": "procedural_thread_3"}}
)

print(f"User: How can I sleep better?")
print(f"\nAssistant (v{version} instructions - after feedback):")
print(response['messages'][-1].content)
print("\nNotice: The response should now be more concise based on the feedback!")

User: How can I sleep better?

Assistant (v2 instructions - after feedback):
Improving your sleep can have a significant impact on your overall wellness. Here are three actionable tips to help you sleep better:

1. **Establish a Consistent Sleep Schedule**: Try to go to bed and wake up at the same time every day, even on weekends. This helps regulate your body's internal clock.

2. **Create a Relaxing Bedtime Routine**: Engage in calming activities before bed, such as reading, gentle stretching, or meditation. Avoid screens for at least 30 minutes before sleep, as blue light can interfere with melatonin production.

3. **Optimize Your Sleep Environment**: Make your bedroom conducive to sleep by keeping it dark, cool, and quiet. Consider using blackout curtains, earplugs, or a white noise machine if needed.

Would you like more specific strategies or information on any of these tips?

Notice: The response should now be more concise based on the feedback!


## Task 10: Unified Wellness Memory Agent

Now let's combine **all 5 memory types** into a unified wellness agent:

1. **Short-term**: Remembers current conversation (checkpointer)
2. **Long-term**: Stores user profile across sessions (store + namespace)
3. **Semantic**: Retrieves relevant wellness knowledge (store + embeddings)
4. **Episodic**: Uses past successful interactions as examples (store + search)
5. **Procedural**: Adapts behavior based on feedback (store + reflection)

### Memory Retrieval Flow

```
User Query: "What exercises can help my back pain?"
              │
              ▼
┌─────────────────────────────────────────────────┐
│  1. PROCEDURAL: Get current instructions         │
│  2. LONG-TERM: Load user profile (conditions)    │
│  3. SEMANTIC: Search wellness knowledge          │
│  4. EPISODIC: Find similar past interactions     │
│  5. SHORT-TERM: Include conversation history     │
└─────────────────────────────────────────────────┘
              │
              ▼
        Generate personalized, informed response
```

In [34]:
class UnifiedState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str
    feedback: str


def unified_wellness_assistant(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """An assistant that uses all five memory types."""
    user_id = state["user_id"]
    user_message = state["messages"][-1].content
    
    # 1. PROCEDURAL: Get current instructions
    instructions_item = store.get(("agent", "instructions"), "wellness_assistant")
    base_instructions = instructions_item.value["instructions"] if instructions_item else "You are a helpful wellness assistant."
    
    # 2. LONG-TERM: Get user profile
    profile_items = list(store.search((user_id, "profile")))
    pref_items = list(store.search((user_id, "preferences")))
    profile_text = "\n".join([f"- {p.key}: {p.value}" for p in profile_items]) if profile_items else "No profile stored."
    
    # 3. SEMANTIC: Search for relevant knowledge
    relevant_knowledge = store.search(("wellness", "knowledge"), query=user_message, limit=2)
    knowledge_text = "\n".join([f"- {r.value['text'][:200]}..." for r in relevant_knowledge]) if relevant_knowledge else "No specific knowledge found."
    
    # 4. EPISODIC: Find similar past interactions
    similar_episodes = store.search(("agent", "episodes"), query=user_message, limit=1)
    if similar_episodes:
        ep = similar_episodes[0].value
        episode_text = f"Similar past interaction:\nUser: {ep.get('input', 'N/A')}\nResponse style: {ep.get('feedback', 'N/A')}"
    else:
        episode_text = "No similar past interactions found."
    
    # Build comprehensive system message
    system_message = f"""{base_instructions}

=== USER PROFILE ===
{profile_text}

=== RELEVANT WELLNESS KNOWLEDGE ===
{knowledge_text}

=== LEARNING FROM EXPERIENCE ===
{episode_text}

Use all of this context to provide the best possible personalized response."""
    
    # 5. SHORT-TERM: Full conversation history is automatically managed by the checkpointer
    # Use summarization for long conversations
    trimmed_messages = summarize_conversation(state["messages"], max_messages=6)
    
    messages = [SystemMessage(content=system_message)] + trimmed_messages
    response = llm.invoke(messages)
    return {"messages": [response]}


def unified_feedback_node(state: UnifiedState, config: RunnableConfig, *, store: BaseStore):
    """Update procedural memory based on feedback."""
    feedback = state.get("feedback", "")
    if not feedback:
        return {}
    
    item = store.get(("agent", "instructions"), "wellness_assistant")
    if item is None:
        return {}
    
    current = item.value
    reflection_prompt = f"""Update these instructions based on feedback:

Current: {current['instructions']}
Feedback: {feedback}

Output only the updated instructions."""
    
    response = llm.invoke([HumanMessage(content=reflection_prompt)])
    store.put(
        ("agent", "instructions"),
        "wellness_assistant",
        {"instructions": response.content, "version": current["version"] + 1}
    )
    print(f"Procedural memory updated to v{current['version'] + 1}")
    return {}


def unified_route(state: UnifiedState) -> str:
    return "feedback" if state.get("feedback") else "end"


# Build the unified graph
unified_builder = StateGraph(UnifiedState)
unified_builder.add_node("assistant", unified_wellness_assistant)
unified_builder.add_node("feedback", unified_feedback_node)

unified_builder.add_edge(START, "assistant")
unified_builder.add_conditional_edges("assistant", unified_route, {"feedback": "feedback", "end": END})
unified_builder.add_edge("feedback", END)

# Compile with both checkpointer (short-term) and store (all other memory types)
unified_graph = unified_builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("Unified wellness assistant ready with all 5 memory types!")

Unified wellness assistant ready with all 5 memory types!


In [35]:
# Test the unified assistant
config = {"configurable": {"thread_id": "unified_thread_1"}}

# First interaction - should use semantic + long-term + episodic memory
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="What exercises would you recommend for my back?")],
        "user_id": "user_sarah",  # Sarah has a bad knee in her profile!
        "feedback": ""
    },
    config
)

print("User: What exercises would you recommend for my back?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\n" + "="*60)
print("Memory types used:")
print("  Long-term: Knows Sarah has a bad knee")
print("  Semantic: Retrieved back exercise info from knowledge base")
print("  Episodic: May use similar joint pain episode as reference")
print("  Procedural: Following current instructions")
print("  Short-term: Will remember this in follow-up questions")

User: What exercises would you recommend for my back?

Assistant: It's great that you're looking to support your back health! Here are three gentle exercises that can help relieve lower back pain:

1. **Cat-Cow Stretch**: Start on your hands and knees. Alternate between arching your back up (like a cat) and letting it sag down (like a cow). Aim for 10-15 repetitions to help increase flexibility and relieve tension.

2. **Bird-Dog**: From the same hands-and-knees position, extend one arm forward and the opposite leg back, keeping your back straight. Hold for a few seconds, then switch sides. This helps strengthen your core and lower back.

3. **Child’s Pose**: Kneel on the floor, sit back on your heels, and stretch your arms forward on the ground. This pose gently stretches the back and can provide relief.

Make sure to listen to your body and stop if you feel any pain. Have you tried any of these exercises before, or do you have any specific concerns about your back?

Memory types used

In [36]:
# Follow-up question (tests short-term memory)
response = unified_graph.invoke(
    {
        "messages": [HumanMessage(content="Can you show me how to do the first one?")],
        "user_id": "user_sarah",
        "feedback": ""
    },
    config  # Same thread
)

print("User: Can you show me how to do the first one?")
print(f"\nAssistant: {response['messages'][-1].content}")
print("\nNotice: The agent remembers the context from the previous message!")

User: Can you show me how to do the first one?

Assistant: Absolutely! Here’s how to do the Cat-Cow Stretch step-by-step:

1. **Start Position**: Begin on your hands and knees on a comfortable surface, like a yoga mat. Your wrists should be directly under your shoulders, and your knees should be under your hips.

2. **Cat Pose**: Inhale deeply. As you exhale, round your back towards the ceiling, tucking your chin to your chest and drawing your belly button in towards your spine. This is the "cat" position.

3. **Cow Pose**: Inhale again. As you exhale, arch your back, letting your belly drop towards the floor while lifting your head and tailbone towards the ceiling. This is the "cow" position.

4. **Repeat**: Continue to alternate between these two positions for 10-15 repetitions, coordinating your breath with your movements.

Remember to move slowly and gently, focusing on your breath. How does that sound? Would you like tips on how to incorporate this into your routine?

Notice: The 

---
## ❓ Question #3:

How would you decide what constitutes a **"successful" wellness interaction** worth storing as an episode? What metadata should you store alongside the episode?

Consider:
- Explicit feedback (thumbs up) vs implicit signals
- User engagement (did they ask follow-up questions?)
- Objective outcomes vs subjective satisfaction
- Privacy implications of storing interaction data

##### Answer:
Criteria for Successful Interaction
1. Explicit Signals (High Confidence)
User gives a thumbs up
 Positive rating (4–5 stars)
 Explicit comment: “This was really helpful!”  
 User saves/bookmarks the response

2. Implicit Signals (Medium Confidence)
 Deep engagement:
2+ follow‑up questions
Reading time >30 seconds
User applied the advice (if trackable)

Completion signals:
User completed the suggested plan
Returned to update progress
Asked for related advice

No abandonment:
Did not close immediately after the response
Did not reformulate the question (which would indicate dissatisfaction)

3. Objective Outcomes (Most Reliable)
 Measurable metrics:
User achieved the goal (lost weight, slept better)
Wellness metrics improved (app tracking)

 Repeated behavior:
User returned for similar advice
User recommended it to others

Privacy Considerations
CRITICAL for Wellness Systems

PII Removal
Remove names, exact dates, and specific locations
Hash user_ids
Generalize medical conditions (e.g., “joint pain” instead of “right‑knee arthritis”)

Consent Management
Obtain explicit consent to store interactions
Allow opt‑out at any time
Support the right to be forgotten (GDPR)

Data Minimization
Store only what is necessary for few‑shot learning
Do not store sensitive medical data unless anonymized
Apply automatic TTL (e.g., 6 months)

Anonymization

Recommended Strategy
Level 1 – Automatic (Score ≥ 70)
Thumbs up + follow‑ups + goal achieved
Automatic storage

Level 2 – Review (Score 50–69)
Mixed signals
Human‑in‑the‑loop review before storing

Level 3 – Discarded (Score < 50)
Insufficient success signals
Do not store

## ❓ Question #4:

For a **production wellness assistant**, which memory types need persistent storage (PostgreSQL) vs in-memory? How would you handle memory across multiple agent instances (e.g., Exercise Agent, Nutrition Agent, Sleep Agent)?

Consider:
- Which memories are user-specific vs shared?
- Consistency requirements across agents
- Memory expiration and cleanup policies
- Namespace strategy for multi-agent systems

##### Answer:

Aspect	                            Decision	                               reason
Short‑term storage	       PostgreSQL	                           Persistence across sessions
Long‑term storage	       PostgreSQL	                           CRITICAL – user data
Semantic storage	       Vector DB + PostgreSQL	               Efficient search + persistence
Episodic storage	       PostgreSQL	                           Continuous learning
Procedural storage	        PostgreSQL	                           Instruction versioning
Hot data cache	            Redis	                               Performance for active users
Namespace strategy	        User‑specific + Shared	               Isolation + shared knowledge
Consistency	                for profiles,Eventual for episodes	   Security/performance balance
TTL policies	            Variable by type	                   GDPR compliance + optimized cost
Multi‑agent coordination	Event bus + Shared store	           Efficient synchronization

---
## 🏗️ Activity #2: Wellness Memory Dashboard

Build a wellness tracking system that:
1. Tracks wellness metrics over time (mood, energy, sleep quality)
2. Uses semantic memory to find relevant advice
3. Uses episodic memory to recall what worked before
4. Uses procedural memory to adapt advice style
5. Provides a synthesized "wellness summary"

### Requirements:
- Store at least 3 wellness metrics per user
- Track metrics over multiple "days" (simulated)
- Agent should reference historical data in responses
- Generate a personalized wellness summary

In [ ]:
# ============================================================================
# Activity #2: Wellness Memory Dashboard - Notebook Version
# ============================================================================

from typing import List, Dict
from datetime import datetime, timedelta


# ============================================================================
# Define Wellness Metrics Schema and Storage Functions
# ============================================================================

def log_wellness_metric(store, user_id: str, date: str, metric_type: str, value: float, notes: str = ""):
    """Log a wellness metric for a user on a specific date."""
    metric_namespace = (user_id, "wellness_metrics")
    key = f"{date}_{metric_type}"
    
    store.put(
        metric_namespace,
        key,
        {
            "date": date,
            "metric_type": metric_type,
            "value": value,
            "notes": notes,
            "timestamp": datetime.now().isoformat()
        }
    )


def get_wellness_history(store, user_id: str, metric_type: str = None, days: int = 7) -> List[Dict]:
    """Get wellness history for a user."""
    metric_namespace = (user_id, "wellness_metrics")
    all_metrics = list(store.search(metric_namespace))
    
    cutoff_date = (datetime.now() - timedelta(days=days)).strftime("%Y-%m-%d")
    
    filtered = []
    for item in all_metrics:
        metric_data = item.value
        if metric_data["date"] >= cutoff_date:
            if metric_type is None or metric_data["metric_type"] == metric_type:
                filtered.append(metric_data)
    
    filtered.sort(key=lambda x: x["date"], reverse=True)
    return filtered


def calculate_metric_stats(metrics: List[Dict]) -> Dict:
    """Calculate statistics for a list of metrics."""
    if not metrics:
        return {}
    
    values = [m["value"] for m in metrics]
    
    return {
        "avg": sum(values) / len(values),
        "min": min(values),
        "max": max(values),
        "latest": values[0],
        "trend": "improving" if len(values) > 1 and values[0] > values[-1] else 
                 "declining" if len(values) > 1 and values[0] < values[-1] else
                 "stable"
    }


def format_wellness_history(store, user_id: str, days: int = 7) -> str:
    """Format wellness history for display in prompt."""
    lines = [f"WELLNESS HISTORY (Last {days} days):"]
    
    for metric_name in ["mood", "energy", "sleep_quality", "stress_level"]:
        history = get_wellness_history(store, user_id, metric_name, days)
        
        if history:
            stats = calculate_metric_stats(history)
            lines.append(f"\n{metric_name.replace('_', ' ').title()}:")
            lines.append(f"  - Latest: {stats['latest']:.1f}/10")
            lines.append(f"  - Average: {stats['avg']:.1f}/10")
            lines.append(f"  - Trend: {stats['trend']}")
    
    return "\n".join(lines)


In [24]:
# ============================================================================
#  Create Sample Wellness Data (Simulate a Week)
# ============================================================================

print("Creating sample wellness data...")

base_date = datetime.now() - timedelta(days=7)

# User 1: Sarah - Improving sleep
for day in range(7):
    date = (base_date + timedelta(days=day)).strftime("%Y-%m-%d")
    log_wellness_metric(semantic_store, "user_sarah", date, "sleep_quality", 5 + day * 0.5)
    log_wellness_metric(semantic_store, "user_sarah", date, "energy", 5 + day * 0.3)
    log_wellness_metric(semantic_store, "user_sarah", date, "mood", 6 + day * 0.2)
    log_wellness_metric(semantic_store, "user_sarah", date, "stress_level", 7 - day * 0.3)

print("✓ Sarah: 7 days logged")

# User 2: Mike - Inconsistent exercise
for day in range(7):
    date = (base_date + timedelta(days=day)).strftime("%Y-%m-%d")
    exercise = 45 if day % 2 == 0 else 0
    log_wellness_metric(semantic_store, "user_mike", date, "exercise_minutes", exercise)
    log_wellness_metric(semantic_store, "user_mike", date, "energy", 7 if exercise > 0 else 4)
    log_wellness_metric(semantic_store, "user_mike", date, "mood", 7 if exercise > 0 else 5)
    log_wellness_metric(semantic_store, "user_mike", date, "sleep_quality", 7)

print("✓ Mike: 7 days logged")

# User 3: Emma - High stress affecting sleep
for day in range(7):
    date = (base_date + timedelta(days=day)).strftime("%Y-%m-%d")
    log_wellness_metric(semantic_store, "user_emma", date, "stress_level", 8 - day * 0.1)
    log_wellness_metric(semantic_store, "user_emma", date, "sleep_quality", 4 + day * 0.2)
    log_wellness_metric(semantic_store, "user_emma", date, "energy", 4 + day * 0.1)
    log_wellness_metric(semantic_store, "user_emma", date, "mood", 5 - day * 0.1)

print("✓ Emma: 7 days logged\n")



Creating sample wellness data...
✓ Sarah: 7 days logged
✓ Mike: 7 days logged
✓ Emma: 7 days logged



In [25]:
# ============================================================================
# Build Wellness Dashboard Agent
# ============================================================================

class DashboardState(TypedDict):
    messages: Annotated[list, add_messages]
    user_id: str


def wellness_dashboard_agent(state: DashboardState, config: RunnableConfig, *, store: BaseStore):
    """Dashboard agent using ALL 5 memory types."""
    user_id = state["user_id"]
    user_question = state["messages"][-1].content
    
    # 1. LONG-TERM: User profile
    profile_namespace = (user_id, "profile")
    profile_items = list(store.search(profile_namespace))
    profile_context = "USER PROFILE:\n" + "\n".join([f"- {p.key}: {p.value.get('value')}" for p in profile_items]) if profile_items else "No profile"
    
    # 2. TIME-SERIES: Wellness metrics
    wellness_history = format_wellness_history(store, user_id, days=7)
    
    # 3. SEMANTIC: Wellness knowledge
    kb_results = store.search(("wellness", "knowledge"), query=user_question, limit=2)
    knowledge_context = "\nKNOWLEDGE:\n" + "\n".join([f"- {r.value.get('text', '')[:100]}..." for r in kb_results]) if kb_results else "No knowledge found"
    
    # 4. EPISODIC: Past successful cases
    episodic_results = store.search(("agent", "episodes"), query=user_question, limit=1)
    episodic_context = ""
    if episodic_results:
        ep = episodic_results[0].value
        episodic_context = f"\nPAST SUCCESS:\nSituation: {ep.get('situation', '')}\nApproach: {ep.get('output', '')[:150]}..."
    
    # 5. PROCEDURAL: Current instructions
    proc_item = store.get(("agent", "instructions"), "wellness_assistant")
    instructions = proc_item.value.get("instructions", "You are a wellness assistant.") if proc_item else "You are a wellness assistant."
    
    # Build comprehensive prompt
    system_prompt = f"""{instructions}

{profile_context}

{wellness_history}

{knowledge_context}

{episodic_context}

TASK: Analyze wellness metrics and provide:
1. Key insights about patterns
2. Specific recommendations based on data
3. Encouragement for positive trends
Reference actual metrics in your response."""
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}


# Build dashboard graph
dashboard_builder = StateGraph(DashboardState)
dashboard_builder.add_node("dashboard", wellness_dashboard_agent)
dashboard_builder.add_edge(START, "dashboard")
dashboard_builder.add_edge("dashboard", END)

wellness_dashboard = dashboard_builder.compile(
    checkpointer=MemorySaver(),
    store=semantic_store
)

print("✓ Wellness dashboard agent compiled\n")



✓ Wellness dashboard agent compiled



In [26]:
# ============================================================================
#  Generate Wellness Summary
# ============================================================================

def generate_wellness_summary(store, user_id: str, days: int = 7) -> str:
    """Generate comprehensive wellness summary."""
    lines = [f"WELLNESS SUMMARY - Last {days} Days", "=" * 60]
    
    metrics_by_type = {}
    for metric_type in ["mood", "energy", "sleep_quality", "stress_level"]:
        history = get_wellness_history(store, user_id, metric_type, days)
        if history:
            metrics_by_type[metric_type] = history
    
    if not metrics_by_type:
        return "No wellness data available."
    
    lines.append("\n📊 OVERALL TRENDS:")
    for metric_type, history in metrics_by_type.items():
        stats = calculate_metric_stats(history)
        trend_emoji = "📈" if stats["trend"] == "improving" else "📉" if stats["trend"] == "declining" else "➡️"
        
        lines.append(f"\n{metric_type.replace('_', ' ').title()}:")
        lines.append(f"  {trend_emoji} {stats['trend'].title()}")
        lines.append(f"  Current: {stats['latest']:.1f}/10")
        lines.append(f"  Average: {stats['avg']:.1f}/10")
    
    # Key insights
    lines.append("\n\n🔍 KEY INSIGHTS:")
    
    if "sleep_quality" in metrics_by_type and "energy" in metrics_by_type:
        sleep_stats = calculate_metric_stats(metrics_by_type["sleep_quality"])
        energy_stats = calculate_metric_stats(metrics_by_type["energy"])
        
        if sleep_stats["trend"] == "improving" and energy_stats["trend"] == "improving":
            lines.append("  ✓ Sleep improvements correlating with better energy")
    
    if "stress_level" in metrics_by_type:
        stress_stats = calculate_metric_stats(metrics_by_type["stress_level"])
        if stress_stats["avg"] > 7:
            lines.append("  ⚠ Stress levels have been elevated")
        elif stress_stats["trend"] == "declining":
            lines.append("  ✓ Stress is declining - keep it up!")
    
    return "\n".join(lines)



In [27]:
# ============================================================================
# Test the Dashboard
# ============================================================================

print("="*70)
print("TESTING WELLNESS DASHBOARD")
print("="*70)

# Test 1: Sarah's Summary
print("\n📊 Sarah's Wellness Summary:")
print("-"*70)
summary = generate_wellness_summary(semantic_store, "user_sarah", days=7)
print(summary)

# Test 2: Sarah's Insights
print("\n\n💬 Sarah's Personalized Insights:")
print("-"*70)
config = {"configurable": {"thread_id": "sarah_dash_1"}}

response = wellness_dashboard.invoke(
    {
        "messages": [HumanMessage(content="Give me insights about my wellness this week")],
        "user_id": "user_sarah"
    },
    config
)

print(f"\nAgent: {response['messages'][-1].content}")

# Test 3: Mike's Question
print("\n\n💬 Mike's Energy Question:")
print("-"*70)
config2 = {"configurable": {"thread_id": "mike_dash_1"}}

response2 = wellness_dashboard.invoke(
    {
        "messages": [HumanMessage(content="I've been feeling tired lately. What might help?")],
        "user_id": "user_mike"
    },
    config2
)

print(f"\nAgent: {response2['messages'][-1].content}")

# Test 4: Emma's Summary
print("\n\n📊 Emma's Wellness Summary:")
print("-"*70)
summary = generate_wellness_summary(semantic_store, "user_emma", days=7)
print(summary)

print("\n" + "="*70)
print("✓ Activity #2 Complete!")
print("="*70)
print("\nMemory Types Demonstrated:")
print("  ✓ SHORT-TERM: Conversation history")
print("  ✓ LONG-TERM: User profiles")
print("  ✓ SEMANTIC: Wellness knowledge")
print("  ✓ EPISODIC: Past successes")
print("  ✓ PROCEDURAL: Dashboard instructions")


TESTING WELLNESS DASHBOARD

📊 Sarah's Wellness Summary:
----------------------------------------------------------------------
WELLNESS SUMMARY - Last 7 Days

📊 OVERALL TRENDS:

Mood:
  📈 Improving
  Current: 6.2/10
  Average: 6.1/10

Energy:
  📈 Improving
  Current: 5.6/10
  Average: 5.3/10

Sleep Quality:
  📈 Improving
  Current: 6.0/10
  Average: 5.5/10

Stress Level:
  📉 Declining
  Current: 6.7/10
  Average: 6.8/10


🔍 KEY INSIGHTS:
  ✓ Sleep improvements correlating with better energy
  ✓ Stress is declining - keep it up!


💬 Sarah's Personalized Insights:
----------------------------------------------------------------------

Agent: Based on your wellness metrics from the last 7 days, here are some key insights:

### Key Insights:
1. **Mood Improvement**: Your latest mood score is 6.2/10, with an average of 6.1/10. This indicates a slight but positive trend in your mood, suggesting that you may be experiencing more positive emotions or coping better with challenges.

2. **Energy

In [ ]:
### YOUR CODE HERE ###

# Step 1: Define wellness metrics schema and storage functions
def log_wellness_metric(store, user_id: str, date: str, metric_type: str, value: float, notes: str = ""):
    """Log a wellness metric for a user."""
    pass


def get_wellness_history(store, user_id: str, metric_type: str = None, days: int = 7) -> list:
    """Get wellness history for a user."""
    pass


# Step 2: Create sample wellness data for a user (simulate a week)


# Step 3: Build a wellness dashboard agent that:
#   - Retrieves user's wellness history
#   - Searches for relevant advice based on patterns
#   - Uses episodic memory for what worked before
#   - Generates a personalized summary


# Step 4: Test the dashboard
# Example: "Give me a summary of my wellness this week"
# Example: "I've been feeling tired lately. What might help?"


---
## Summary

In this session, we explored the **5 memory types** from the CoALA framework:

| Memory Type | LangGraph Component | Scope | Wellness Use Case |
|-------------|---------------------|-------|-------------------|
| **Short-term** | `MemorySaver` + `thread_id` | Within thread | Current consultation |
| **Long-term** | `InMemoryStore` + namespaces | Across threads | User profile, goals |
| **Semantic** | Store + embeddings + `search()` | Across threads | Knowledge retrieval |
| **Episodic** | Store + few-shot examples | Across threads | Past successful interactions |
| **Procedural** | Store + self-reflection | Across threads | Self-improving instructions |

### Key Takeaways:

1. **Memory transforms chatbots into assistants** - Persistence enables personalization
2. **Different memory types serve different purposes** - Choose based on your use case
3. **Context management is critical** - Trim and summarize to stay within limits
4. **Episodic memory enables learning** - Show, don't just tell
5. **Procedural memory enables adaptation** - Agents can improve themselves

### Production Considerations:

- Use `PostgresSaver` instead of `MemorySaver` for persistent checkpoints
- Use `PostgresStore` instead of `InMemoryStore` for persistent long-term memory
- Consider TTL (Time-to-Live) policies for automatic memory cleanup
- Implement proper access controls for user data

### Further Reading:

- [LangGraph Memory Documentation](https://langchain-ai.github.io/langgraph/concepts/memory/)
- [CoALA Paper](https://arxiv.org/abs/2309.02427) - Cognitive Architectures for Language Agents
- [LangGraph Platform](https://docs.langchain.com/langgraph-platform/) - Managed infrastructure for production